### Instalando las librerias necesarias

Para la correcta ejecucion del proyecto primero se deberan de instalar las librerias usadas para la implementacion del mismo.

In [110]:
!pip install anytree pandas
!pip install pytholog
!apt install swi-prolog



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



### Implementacion del agente

In [111]:
import numpy as np
import pandas as pd
from anytree import NodeMixin, RenderTree
import pytholog as pl 
#from pyswip import Prolog

In [112]:
class Task:
    def __init__(self, id, taskType, name, avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves):
        self.id=id
        self.taskType=taskType
        self.name=name
        self.avgTime=avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
    
    def __str__(self) -> str:
        """
        To String del objeto Task. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Task): Instancia de la clase Task.
        
        Returns:
            str: Representacion como texto del objeto Task.
        """
        return f"Name: {self.name}, AvgTime: {self.avgTime}, Urgency: {self.urgency}"



class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self,taskType, name,  avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves, parent=None, children=None):
        super(Task, self).__init__()
        self.taskType=taskType
        self.name = name
        self.avgTime = avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
        self.parent = parent
        if children:
            self.children = children

In [113]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency



    def __str__(self):
        """
        To String del objeto Choice. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Choice): Instancia de la clase Choice.
        
        Returns:
            str: Representacion como texto del objeto Choice.
        """
        return self.move + ": " + str(self.urgency)
    

In [114]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características


class ScheduleAgent:
  def __init__(self, name:str, age:int, gender:str, taskList:pd.DataFrame, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.taskList = taskList
    self.notification = comments
    self.comments = comments
    self.rootNode = self.buildTree() #creo el arbol al inicializar el objeto
    
    
  
  def getNodesList(self) -> list:
    """
    Crear una lista de nodos del arbol de tareas de acuerdo a las
    tareas almacenadas en un archivo CSV.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        list: Lista de nodos del arbol de tareas, cada nodo representa una tarea.
    """

# TaskID,TaskType,Name,AvgTime,TimeStart,Deadline,Urgency,location,isHobby,recurrency,bestOn,involves,Parent,Children

    nodeList = []
    for i in range(0, len(self.taskList["Name"])):
        nodeList.append(TaskClass(
            taskType = self.taskList["TaskType"][i],
            name = self.taskList["Name"][i],
            avgTime = self.taskList["AvgTime"][i],
            timeStart = self.taskList["TimeStart"][i],
            deadline = self.taskList["Deadline"][i],
            urgency = self.taskList["Urgency"][i],
            location = self.taskList["Location"][i],
            isHobby = self.taskList["IsHobby"][i],
            recurrency = self.taskList["Recurrency"][i],
            bestOn = self.taskList["BestOn"][i],
            involves = self.taskList["Involves"][i],
        ))
    return nodeList
  
  
  
  def setParentNodes(self, nodeList:list) -> TaskClass:
    """
    Colocar a cada uno de los nodos del arbol de tareas su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        nodeList (list): Lista de nodos del arbol de tareas, cada nodo representa una tarea.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(self.taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

  
  
  def buildTree(self) -> TaskClass: 
    """
    Armar el arbol de tareas.
    En primera instancia crea una lista de nodos de tareas para 
    despues asignarles su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    nodeList = self.getNodesList()
    rootNode = self.setParentNodes(nodeList)
    return rootNode #retorno root del arbol ya que desde root lo puedo recorrer todo
    
    
    
  def printTaskTree(self) -> None:
    """
    Imprimir por completo el arbol de tareas. 
    
    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
    """
    for pre, fill, node in RenderTree(self.rootNode):
        print("%s%s" % (pre, node.name))   
  
  
  
  def miniMax(self, rootNode:Task, isMax:bool) -> Choice:
    """
    Algoritmo de busqueda miniMax para determinar la mayor cantidad de tareas de la mayor urgencia posible
    a completar en el minimo tiempo posible.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        rootNode (Task): Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        isMax (bool): ?

    Returns:
        Choice: Eleccion de rama y nodo del algortimo miniMax
    """
    children = rootNode.children
    try:
        l_choice = self.miniMax(children[0], not isMax)
        r_choice = self.miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", self.rootNode.urgency)

In [115]:
#inicializando el agente para el usuario Ricardo
myAgent = ScheduleAgent(name="Ricardo",
                        age=36,
                        gender="Male",
                        taskList=pd.read_csv('taskList.csv'),
                        comments = "")

isMax = True
currentNode = myAgent.rootNode

print("Comenzamos nuestro recorrido en el nodo: "+str(currentNode.name)+", con una urgencia de: "+str(currentNode.urgency))

while (True):
    # run minimax on current node
    agentChoice = myAgent.miniMax(currentNode, isMax) 
    
    # make choice based on minimax search
    if (agentChoice.move == "left"):
        print ("Nos movemos a la izquierda con una urgencia de: " + str(currentNode.children[0].urgency)+ ", a la tarea: "+str(currentNode.children[0].name))
        currentNode = currentNode.children[0]
    elif (agentChoice.move == "right"):
        print ("Nos movemos a la derecha con una urgencia de: " + str(currentNode.children[1].urgency)+ ", a la tarea: "+str(currentNode.children[1].name))
        currentNode = currentNode.children[1]
    elif (agentChoice.move == "end"):
        print ("Hemos llegado al final del mejor recorrido")
        break

Comenzamos nuestro recorrido en el nodo: desayunar, con una urgencia de: 5
Nos movemos a la derecha con una urgencia de: 5, a la tarea: sacar al perro
Nos movemos a la derecha con una urgencia de: 7, a la tarea: regar plantas
Nos movemos a la derecha con una urgencia de: 6, a la tarea: jugar con el perro
Hemos llegado al final del mejor recorrido


In [116]:
# PARA ORGANIZAR LAS TAREAS SEGUN SUS ATRIBUTOS.

dictTasks = {}
dictHobbies = {}
dictHabits = {}

In [117]:
# LISTAR INFORMACION

def listName(data):
  list = []
  for key, value in data.items():
    temp = key
    list.append(temp)
  return list

def listIsHabit(data):
  list = []
  for key, value in data.items():
    temp = value["isHabit"]
    list.append(temp)
  return list

def listIsHobby(data):
  list = []
  for key, value in data.items():
    temp = value["isHobby"]
    list.append(temp)
  return list


listaHobbies = listIsHobby(dictHobbies)
listaHbitos = listIsHabit(dictHabits)

In [2]:
global KB 
KB = pl.knowledge_base("tasks")

def baseConocimiento():
  facts = []
  
  #rules
  facts.append("isTask(X)")
  facts.append("isHobby(X,Y):- isTask(X), Y=True")
  facts.append("isHabit(X,Y):- isTask(X), Y>5")
  facts.append("isHabit(X,Y):- isTask(X), Y>5") 
  facts.append("userSchedule(X):- isTask(X), isHobby(X), isHabit(X)") 
  facts.append("builBaseConfiguration(X):- userSchedule(X)") 
  facts.append("patternIdentifier(X):- userSchedule(X), buildBaseConfiguration(X)") 
  facts.append("suggestedActivities(X,Y):- isHabit(X) => 3, patternIdentifier(X)")
  # REFERENTES
  #facts.append("hayProducto(X):- hay(X, Y), Y > 0")
  #facts.append("alertaProducto(X):- hay(X, Y), Y  <= 6")
  #facts.append("faltaProducto(X):- hay(X, Y), Y  < 1")
  #facts.append("vendio(X,Y,Z):- vende(X,Y,Z), vendedor(X)")
  #facts.append("premiarVendedor(X):- vendedor(X), recompensa(X,Y), Y >= 15")
  #facts.append("castigarVendedor(X):- recompensa(X,Y), vendedor(X), Y <= 5 ")

  #prolog
  
  KB(facts)
  
  #query
  print(facts)

  # REFERENTES
  #print(inventary_kb.query(pl.Expr("alertaProducto(salsa de tomate)")))
  #print(inventary_kb.query(pl.Expr("vendio(tomas, papas, X)")))
  #print(inventary_kb.query(pl.Expr("premiarVendedor(X)")))
  #print(inventary_kb.query(pl.Expr("castigarVendedor(X)")))

baseConocimiento()
  

SyntaxError: invalid syntax (3031018288.py, line 11)

In [119]:
# AÑADIR LAS TAREAS DE LA ESTRUCTURA ANYTREE AL ARCHIVO DE LOGICA DE PROLOG

# METODO PARA AÑADIR LOS TASKS
def addAllTasks(node):
    for node in node.descendants:
        if isinstance(node, Task):
            dictTasks[node.name] = node


# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HOBBIES
def addHobbies(node):
    for node in node.descendants:
        if isinstance(node, Task):
            if node.isHobby == True: 
                dictHobbies[node.name] = node


# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HABITOS
def addHabits(node):
    for node in node.descendants:
        if isinstance(node, Task):
            if node.recurrency >= 5: 
                dictHabits[node.name] = node

addAllTasks(myAgent.rootNode)
addHobbies(myAgent.rootNode)
addHabits(myAgent.rootNode)
print("Tasks: " + str(dictTasks))
print("Hobbies: " + str(dictHobbies))
print("Habits: " + str(dictHabits))

Tasks: {'bañarse': <__main__.TaskClass object at 0x11740cd00>, 'hacer la tarea': <__main__.TaskClass object at 0x11740e920>, 'almorzar': <__main__.TaskClass object at 0x11740d6f0>, 'hacer ejercicio': <__main__.TaskClass object at 0x11740c640>, 'estudiar para el parcial': <__main__.TaskClass object at 0x11740ffd0>, 'escuchar musica': <__main__.TaskClass object at 0x11740c7c0>, 'leer documentacion': <__main__.TaskClass object at 0x11740f850>, 'sacar al perro': <__main__.TaskClass object at 0x11740f220>, 'hacer estiramientos': <__main__.TaskClass object at 0x11740fc40>, 'ver netflix': <__main__.TaskClass object at 0x11740e380>, 'reunirse con amigos': <__main__.TaskClass object at 0x11740dc90>, 'regar plantas': <__main__.TaskClass object at 0x11740f370>, 'ordenar el cuarto': <__main__.TaskClass object at 0x11740efe0>, 'jugar con el perro': <__main__.TaskClass object at 0x11740ebf0>}
Hobbies: {'escuchar musica': <__main__.TaskClass object at 0x11740c7c0>, 'ver netflix': <__main__.TaskClass 